In [1]:
!pip install aitextgen
!pip install tensorflow

In [2]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Jewish Strawberry Jam And Chocolate Chip Mandelbrot As Made By Danielle Warren

3 cups all-purpose flour
¾ cup granulated sugar
2 teaspoons baking powder
1 cup corn oil
1 teaspoon vanilla extract
3 teaspoon large eggs
1 cup strawberry jam
½ cup semi-sweet chocolate chips
½ cup cinnamon sugar


1. Preheat the oven to 350˚F (180˚C). Line a baking sheet with parchment paper.
2. In a large bowl, whisk together the flour, sugar, and baking powder. Add the corn oil, vanilla, and eggs, and stir with a rubber spatula until the dough starts to come together. Use your hands to finish mixing the dough until well combined and thick.
3. If making strawberry mandelbrot, wet your hands with water and shape half of the dough into a rough oval on the prepared baking sheets. Pinch the edges of the dough up to create a well. Spread the strawberry jam in the center of the dough. Flatten the remaining dough and place on top of the strawberry jam. Leave a bit of extra dough in the bowl to fill in the cracks

In [3]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe2.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [4]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, to_gpu=True, config=config, tf_gpt2="355m")

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#Ai training disabled to prevent accidental overrides.


# file_name = "recipe2.csv"
# ai.save("aitextgen.model")
# train_tokenizer(file_name)
# data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
# ai.train(data, batch_size=32, num_steps=64000, generate_every=1000, save_every=5000)
# ai.generate(prompt="steak")

# #Save the model
# ai.save("aitextgen.model")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.
100%|██████████| 9922/9922 [00:00<00:00, 12852.47it/s]
pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v

  0%|          | 0/64000 [00:00<?, ?it/s]

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\trainer.py:2281: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  rank_zero_deprecation(


1,000 steps reached: generating sample texts.                                               
 pan.                                                                                               
9. Remove the tops from the marinade and spread the filling into the crusts. Spread the chocolate mixture over the chocolate cake, and spread into a spheres.
10. Repeat on the prepared baking sheet.
11. Bake for 10–20 minutes, or until the cake is lightly golden brown and set on top is set. Let cooled.
12.
12.
12. Spread the cake.
12.
12. Top with the cookie chocolate is cooled a spoonfuls for 20 minutes tortilled, or serve.
12 making the cake topping.
12, cover the cookies can be cream, then serve.
12 with guer.
12 cookie backers onto each puer.
12 barbacker.
12 hours, 13. Take for 2 hours.
12 cookie, or for at lefuliscream macarballs cookies, then serve, or let cooled cookie.
12 cookie, then serve.
12 d
2,000 steps reached: generating sample texts.                                              

In [1]:
# Load the trained model
ai = aitextgen(model_folder="trained_model", tokenizer_file="aitextgen.tokenizer.json")
ai.generate(prompt="pizza")

NameError: name 'aitextgen' is not defined